In [2]:
from Uplift_PDF import PDF
from Uplift_PDF import report_data

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from pylab import rcParams
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
rcParams['figure.figsize'] = 12,4
rcParams['savefig.dpi'] = 300
from tqdm import tqdm
logo_path = 'logo/'

now = datetime.now().strftime('%Y%m%d')

day = 20231226
file_list = sorted(glob(f'data/{day}/process/*csv'))
fname = [os.path.basename(i).replace('.csv','') for i in file_list]
fname

/Users/yongseok/opt/anaconda3/envs/ML/lib/python3.11/site-packages/fpdf/__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


['chominseok_002', 'kwangminchoi_005']

In [7]:
class PDF(PDF):
    def header(self):
        # Logo
        self.set_font('helvetica', 'I', 8)
        self.cell(0, -10, 'Kookmin Univ. Hitting Report', ln = True)

    # Page footer
    def footer(self):
        # Set position of the footer
        self.set_y(-15)
        # set font
        self.set_font('helvetica', 'I', 8)
        # Page number
        self.cell(0, 10, f"Page {self.page_no()}" + "/" +f"{{nb}}", align = "C")
        # Logo
        self.image(f'{logo_path}/logo.png',
                   160, 280, 50)
    
    # Adding chapter title to start of each chapter
    def chapter_title(self, ch_num, ch_title):
        # set font
        self.set_font('helvetica', '', 12)
        # background color
        self.set_fill_color(0, 128, 0)
        self.set_text_color(255, 255, 255)
        # Chapter title
        chapter_title = f'Chapter {ch_num} : {ch_title}'
        self.cell(0, 5, chapter_title, ln=1, fill=1)
        # line break
        self.ln()

    # Chapter content
    def chapter_body(self):
        # end each chapter
        self.set_font('helvetica', 'I', 12)
        self.cell(0, 5, 'END OF CHAPTER')

    def print_chapter(self, ch_num, ch_title):
        self.add_page()
        self.chapter_title(ch_num, ch_title)
        self.chapter_body()
        
ks_cols = {
    'pelvis_rotational_velocity_with_respect_to_ground'  : ['Pelvic', 'r'],
    'trunk_rotational_velocity_with_respect_to_ground'   : ['Torso', 'g'],
    'lead_arm_rotational_velocity_with_respect_to_ground': ['Upper Arm','b'],
}

one_cols = {
    'trunk_twist_clockwise'                  : 'X FACTOR',
    'trunk_lateral_flexion'                  : 'TRUNK LATERAL TILT',
    'rear_shoulder_adduction'                : 'REAR SHOULDER AD-ABDUCTION',
    'lead_knee_extension'                    : 'LEAD KNEE FLEXION-EXTENSION',
    'lead_knee_extension_velocity'           : 'LEAD KNEE EXTENSION ANGULAR VELOCITY',
    'lead_hip_flexion_with_respect_to_trunk' : 'LEAD HIP FLEXION-EXTENSION',
    'shank_angle'                            : 'SHANK ANGLE',
    'hip_elbow_loss_space'                   : 'HIP-ELBOW DISTANCE',
    'shoulder_hand_loss_space'               : 'SHOULDER-HAND DISTANCE',
    'hand_shoulder_distance'                 : 'SHOULDER-HAND DISTANCE (AP)',
}

lead_cols= {
    'lead_shoulder_external_rotation' : ['SHOULDER', 'darkblue'],
    'lead_elbow_flexion'              : ['ELBOW'   , 'firebrick'],
}

elbow_cols = {
    'rear_elbow_jc_3d_y' : ['REAR ELBOW', 'darkblue'],
    'lead_elbow_jc_3d_y' : ['LEAD ELBOW', 'firebrick'],
}

for file, name in tqdm(zip(file_list, fname)):
    df = pd.read_csv(file)
    to_time = df['to_frame'].values[0]
    kh_time = df['kh_frame'].values[0] - to_time
    fc_time = df['fc_frame'].values[0] - to_time
    bc_time = df['bc_frame'].values[0]
    pel_initial_time = df['pelvis_rotation_initial'][0] - to_time
    df = df.iloc[to_time:bc_time+60].reset_index(drop=True)
    
    bc_time = bc_time - to_time
    to_time = 0
    time = df['time']
    
    store, _ = os.path.split(file)
    player = name.split('_')[0]
    weight = df['weight'][0]
    height = df['height'][0]
    side = df['handedness'][0]
    
    ks = report_data.kinematic_sequence(df,ks_cols, time, to_time, kh_time, fc_time, bc_time)
    ang = report_data.one_angle(df,one_cols, time, to_time, kh_time, fc_time, bc_time)
    arm = report_data.lead_arm(df,lead_cols, time, to_time, kh_time, fc_time, bc_time)
    elb = report_data.elbow_height(df,elbow_cols, time, to_time, kh_time, fc_time, bc_time)
    
    pdf = PDF("P", "mm","A4")
    pdf.set_auto_page_break(auto = True, margin = 15)
    pdf.add_font(family = 'kor', style = 'B', fname =  'NanumGothicCoding-Bold.ttf', uni=True)
    pdf.add_font(family = 'kor', style = '', fname =  'NanumGothicCoding.ttf', uni=True)
    # ===========================================================================================================
    # main page
    pdf.add_page()
    # pdf.set_fill_color(r=150,g=0,b=0)
    # pdf.rect(w=210, h=297, x=0, y=0, style="DF")
    pdf.set_font('helvetica', 'BU', 20)
    title = 'Kookmin Univ. Hitting Report'
    pdf.ln(10)
    title_w = pdf.get_string_width(title)
    doc_w = pdf.w
    pdf.set_x(title_w/2)
    pdf.cell(135, 20, title, align = 'C', ln = True)

    # ===========================================================================================================
    # chapter
    ch1_link = pdf.add_link()
    ch2_link = pdf.add_link()
    ch3_link = pdf.add_link()
    ch4_link = pdf.add_link()
    ch5_link = pdf.add_link()
    ch6_link = pdf.add_link()
    ch7_link = pdf.add_link()
    ch8_link = pdf.add_link()
    ch9_link = pdf.add_link()
    ch10_link = pdf.add_link()
    ch11_link = pdf.add_link()
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(0, 2,"",ln=1,align = 'C')
    pdf.cell(0, 0, f"Name : {player}    Date : {day}   Hitter Type : {side}   Weight : {weight}    Height : {height}", align = 'C', ln = True)
    pdf.cell(0, 5,"",ln=1,align = '')
    pdf.image(f'{logo_path}/analysis.png',x= 30,w=150,h=75)
    pdf.set_font('helvetica', 'B', 15)
    pdf.cell(0,10,"",ln=1,align = '')
    pdf.set_fill_color(r=150,g=0,b=0)
    pdf.set_text_color(255,255,255)
    pdf.cell(0,7, 'Kinematic Parameters', ln = 1, align = 'C', fill=True,link = ch1_link)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,7,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.cell(0,5, '1. KINEMATIC SEQUENCE', ln = 1, align = '', link = ch1_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '2. X FACTOR', ln = 1, align = '', link = ch2_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '3. TRUNK LATERAL TILT', ln = 1, align = '', link = ch3_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '4. LEAD SHOULDER ROTATION AND ELBOW FLEXION', ln = 1, align = '', link = ch4_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '5. REAR SHOULDER AD/ABDUCTION', ln = 1, align = '', link = ch5_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '6. LEAD LEG KNEE FLEXION/EXTEION', ln = 1, align = '', link = ch6_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '7. LEAD LEG KNEE EXTENTION ANGULAR VELOCITY', ln = 1, align = '', link = ch7_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '8. LEAD LEG HIP FLEXION/EXTENSION', ln = 1, align = '', link = ch8_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '9. LEAD LEG KNEE EXTENTION ANGULAR VELOCITY', ln = 1, align = '', link = ch7_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '10. LEAD LEG HIP FLEXION/EXTENSION', ln = 1, align = '', link = ch8_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '11. SHANK ANGLE', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '12. HIP - ELBOW DISTANCE', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '13. ELBOW HEIGHT', ln = 1, align = '', link = ch10_link)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.cell(0,5, '14. LEAD SHOULDER - LEAD ELBOW DISTANCE', ln = 1, align = '', link = ch10_link)
    # pdf.set_font('helvetica', 'B', 15)
    # pdf.cell(0,15,"",ln=1,align = '')
    # pdf.set_text_color(255,255,255)
    # pdf.cell(0,7, 'FEEDBACK', ln = 1, align = 'C',fill=True, link = ch11_link)
    # pdf.set_text_color(0,0,0)

    # ===============================================================================================
    peak_pel = ks['max']['pelvis_rotational_velocity_with_respect_to_ground']; time_pel = ks['time']['pelvis_rotational_velocity_with_respect_to_ground']
    peak_tor = ks['max']['trunk_rotational_velocity_with_respect_to_ground'];time_tor = ks['time']['trunk_rotational_velocity_with_respect_to_ground']
    peak_upper = ks['max']['lead_arm_rotational_velocity_with_respect_to_ground'];time_upper = ks['time']['lead_arm_rotational_velocity_with_respect_to_ground']
    # peak_fore = ks['max']['lead_shoulder_external_rotation_velocity'];time_fore = ks['time']['lead_shoulder_external_rotation_velocity']
    total_time = bc_time - fc_time
    pel_time = round(100 * (time_pel - fc_time) / total_time, 2)
    tor_time = round(100 * (time_tor - fc_time) / total_time, 2)
    upper_time = round(100 * (time_upper - fc_time) / total_time, 2)
    # fore_time = round(100 * (time_fore - fc_time) / total_time, 2)
    tor_gain = round(peak_tor / peak_pel,2)
    upper_gain = round(peak_upper / peak_tor,2)
    # fore_gain = round(peak_fore / peak_upper,2)

    pdf.add_page()
    pdf.set_link(ch1_link, page = 2)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'KINEMATIC SEQUENCE', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The order and velocity of the pelvis, trunk and arm rotation')
    pdf.image(f'figure/KINEMATIC SEQUENCE.png',x=75,y=28,w=133,h=50)
    pdf.image(f'{logo_path}/kinematic.png',x=17,y=28,w=50,h=60)

    data_as_dict = {"Segment"                 : ["Pelvic"       , "Torso"        ,"Arm"            ],
                    "Minor League"            : ["638 - 780"    , "843 - 988"    ,"1102 - 1327"    ],
                    "Peak Velocity"           : [peak_pel       , peak_tor       ,peak_upper       ],
                    "Timing [BC - FC (100%)]" : [f"{pel_time} %", f"{tor_time} %",f"{upper_time} %"],
                    "Speed Gain"              : [''             , tor_gain       ,upper_gain       ]}
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Kinematic Sequence',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [30,40,35,55,40],
                    x_start='C')
    
    pdf.set_font('helvetica', '', 9)
    pdf.set_y(85)
    pdf.cell(0,7,"",ln=1,align = '')
    pdf.cell(0,5, f"Pelvis Initial Rotation Time [BC - FC (100%)] : {round(100 * (pel_initial_time - fc_time) / total_time,2)} %", ln = 1, align = 'R')

    # ===========================================================================================================
    # X-Factor
    pdf.set_link(ch2_link, page = 2)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'X FACTOR [Check Gate Swing]', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The angle between the trunk and the pelvis with respect to the ground')
    pdf.image(f'figure/X FACTOR.png',x=75,y=160,w=133,h=50)
    pdf.image(f'{logo_path}/xfactor.png',x=17,y=160,w=50,h=55)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['trunk_twist_clockwise']],
                    "at Knee High"        : [ang['kh_time']['trunk_twist_clockwise']],
                    "at Fooc Contact"     : [ang['fc_time']['trunk_twist_clockwise']],
                    "at Ball Contact"     : [ang['bc_time']['trunk_twist_clockwise']]
                    }
    pdf.set_y(221)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'X Factor',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                    "Max"        : [ang['min']['trunk_twist_clockwise']],
                    "Max Timing [BC - FC (100%)]" : [f"{round(100 * (ang['min_time']['trunk_twist_clockwise'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(244)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')
    # =============================================================================
    # Trunk Lataral Tilt
    pdf.add_page()
    pdf.set_link(ch3_link, page = 3)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'TRUNK LATERAL TILT [Check the Loss of Posture]', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'Frontal plane rotation of the trunk relative to the pelvis')
    pdf.image('figure/TRUNK LATERAL TILT.png',x=75,y=32,w=133,h=50)
    pdf.image(f'{logo_path}/lateral tilt.png',x=8,y=32,w=65,h=50)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['trunk_lateral_flexion']],
                    "at Knee High"        : [ang['kh_time']['trunk_lateral_flexion']],
                    "at Fooc Contact"     : [ang['fc_time']['trunk_lateral_flexion']],
                    "at Ball Contact"     : [ang['bc_time']['trunk_lateral_flexion']]
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lateral Trunk Tilt',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                    "Max"        : [ang['max']['trunk_lateral_flexion']],
                    "Max Timing [BC - FC (100%)]" : [f"{round(100 * (ang['max_time']['trunk_lateral_flexion'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(113)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')

    # ===========================================================================================================
    # LEAD SHOULDER ROTATION AND ELBOW FLEXION
    pdf.set_link(ch4_link, page = 3)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD SHOULDER ROTATION & ELBOW FLEXION [Check Casting the Hands]', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'Internal and external rotation of the lead shoulder & sagittal plane rotaion of the forearm relative to the upper arm')
    pdf.image(f'figure/Lead Arm.png',x=75,y=165,w=133,h=50)
    pdf.image(f'{logo_path}/leadarm.png',x=17,y=160,w=50,h=60)

    data_as_dict = {
                    "Segment"             : ["Shoulder", "Elbow"],
                    "at Lead Leg Toe Off" : [arm['to_time']['lead_shoulder_external_rotation'],arm['to_time']['lead_elbow_flexion']],
                    "at Knee High"        : [arm['kh_time']['lead_shoulder_external_rotation'],arm['kh_time']['lead_elbow_flexion']],
                    "at Fooc Contact"     : [arm['fc_time']['lead_shoulder_external_rotation'],arm['fc_time']['lead_elbow_flexion']],
                    "at Ball Contact"     : [arm['bc_time']['lead_shoulder_external_rotation'],arm['bc_time']['lead_elbow_flexion']],
                    }
    pdf.set_y(221)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Shoulder External Rotation and Elbow Flexion',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')
    # ===========================================================================================================
    # REAR SHOHULDER AD/ABDUCTION
    pdf.add_page()
    pdf.set_link(ch5_link, page = 4)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'REAR SHOULDER AD/ABDUCTION [Check the Flying Elbow & Dumping the Barrel]', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'Frontal plane rotation of the arm relative to the trunk')
    pdf.image('figure/REAR SHOULDER AD-ABDUCTION.png',x=75,y=30,w=133,h=50)
    pdf.image(f'{logo_path}/reararm.png',x=8,y=25,w=65,h=50)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['rear_shoulder_adduction']],
                    "at Knee High"        : [ang['kh_time']['rear_shoulder_adduction']],
                    "at Fooc Contact"     : [ang['fc_time']['rear_shoulder_adduction']],
                    "at Ball Contact"     : [ang['bc_time']['rear_shoulder_adduction']]
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Rear Shoulder Abduction',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                    "Min"        : [ang['min']['rear_shoulder_adduction']],
                    "Min Timing [BC - FC (100%)]" : [f"{round(100 * (ang['min_time']['rear_shoulder_adduction'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(113)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')
    # ===================================================================================
    # Lead Knee Flexion
    pdf.set_link(ch6_link, page = 4)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG KNEE FLEXION/EXTENSION', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'Sagittal plane rotation of the shank relative to the thigh')
    pdf.image(f'figure/LEAD KNEE FLEXION-EXTENSION.png',x=75,y=160,w=133,h=50)
    pdf.image(f'{logo_path}/knee angle.png',x=17,y=160,w=50,h=50)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['lead_knee_extension']],
                    "at Knee High"        : [ang['kh_time']['lead_knee_extension']],
                    "at Fooc Contact"     : [ang['fc_time']['lead_knee_extension']],
                    "at Ball Contact"     : [ang['bc_time']['lead_knee_extension']]
                    }
    pdf.set_y(221)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Knee Flexion/Extension',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                    "Max"        : [ang['min']['lead_knee_extension']],
                    "Max Timing [BC - FC (100%)]" : [f"{round(100 * (ang['min_time']['lead_knee_extension'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(244)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')

    # ===========================================================================================================
    # Lead Knee Flexion Velcoity
    pdf.add_page()
    pdf.set_link(ch7_link, page = 5)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG KNEE EXTENSION ANGULAR VELOCITY', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'Sagittal plane angular velocity of the shank relative to the thigh')
    pdf.image('figure/LEAD KNEE EXTENSION ANGULAR VELOCITY.png',x=75,y=30,w=133,h=50)
    pdf.image(f'{logo_path}/knee vel.png',x=8,y=25,w=65,h=60)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['lead_knee_extension_velocity']],
                    "at Knee High"        : [ang['kh_time']['lead_knee_extension_velocity']],
                    "at Fooc Contact"     : [ang['fc_time']['lead_knee_extension_velocity']],
                    "at Ball Contact"     : [ang['bc_time']['lead_knee_extension_velocity']]
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Knee Extension Angular Velocity',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                    "Max"        : [ang['max']['lead_knee_extension_velocity']],
                    "Max Timing [BC - FC (100%)]" : [f"{round(100 * (ang['max_time']['lead_knee_extension_velocity'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(113)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')
    # ===================================================================================
    # Lead Hip Angle
    pdf.set_link(ch8_link, page = 5)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG HIP FLEXION/EXTENSION', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'Sagittal plane rotation of the thigh relative to the trunk')
    pdf.image(f'figure/LEAD HIP FLEXION-EXTENSION.png',x=75,y=160,w=133,h=50)
    pdf.image(f'{logo_path}/hip angle.png',x=17,y=160,w=50,h=50)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['lead_hip_flexion_with_respect_to_trunk']],
                    "at Knee High"        : [ang['kh_time']['lead_hip_flexion_with_respect_to_trunk']],
                    "at Fooc Contact"     : [ang['fc_time']['lead_hip_flexion_with_respect_to_trunk']],
                    "at Ball Contact"     : [ang['bc_time']['lead_hip_flexion_with_respect_to_trunk']]
                    }
    pdf.set_y(221)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Hip Flexion/Extension',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                    "Max"        : [ang['max']['lead_hip_flexion_with_respect_to_trunk']],
                    "Max Timing [BC - FC (100%)]" : [f"{round(100 * (ang['max_time']['lead_hip_flexion_with_respect_to_trunk'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(244)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')
    # ===========================================================================================================
    # Shank Angle
    pdf.add_page()
    pdf.set_link(ch9_link, page = 6)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'SHANK ANGLE [Check the Sway]', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The angle of the longitudinal axis of the shank to the vertical axis in the sagittal plane')
    pdf.image('figure/SHANK ANGLE.png',x=75,y=30,w=133,h=50)
    pdf.image(f'{logo_path}/shank_angle.png',x=8,y=25,w=65,h=60)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['shank_angle']],
                    "at Knee High"        : [ang['kh_time']['shank_angle']],
                    "at Fooc Contact"     : [ang['fc_time']['shank_angle']],
                    "at Ball Contact"     : [ang['bc_time']['shank_angle']]
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Shank Angle (Sway)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                    "Min"        : [ang['min']['shank_angle']],
                    "Min Timing [BC - FC (100%)]" : [f"{round(100 * (ang['min_time']['shank_angle'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(113)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')
    
    # ===================================================================================
    # HIP - ELBOW DISTANCE
    pdf.set_link(ch9_link, page = 6)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'HIP - ELBOW DISTANCE [Check Loss of Space]', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The angle in the horizontal plane between the vector between lead hip and rear hip and elbow position')
    pdf.image(f'figure/HIP-ELBOW DISTANCE.png',x=75,y=160,w=133,h=50)
    pdf.image(f'{logo_path}/HIP-ELBOW DISTANCE.png',x=17,y=160,w=50,h=50)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['hip_elbow_loss_space']],
                    "at Knee High"        : [ang['kh_time']['hip_elbow_loss_space']],
                    "at Fooc Contact"     : [ang['fc_time']['hip_elbow_loss_space']],
                    "at Ball Contact"     : [ang['bc_time']['hip_elbow_loss_space']]
                    }
    pdf.set_y(221)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Hip - Elbow Loss of Space',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                    "Min"        : [ang['min']['hip_elbow_loss_space']],
                    "Min Timing [BC - FC (100%)]" : [f"{round(100 * (ang['min_time']['hip_elbow_loss_space'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(244)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')
    # ===========================================================================================================
    # ELBOW HEIGHT
    pdf.add_page()
    pdf.set_link(ch10_link, page = 7)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'ELBOW HEIGHT [Check the Loss of Space]', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The height of the lead and rear elbow')
    pdf.image('figure/elbow_height.png',x=75,y=30,w=133,h=50)
    pdf.image(f'{logo_path}/elbow_height.png',x=8,y=25,w=65,h=60)

    data_as_dict = {
                    "Segment"             : ["Rear Elbow"                        , "Lead Elbow"],
                    "at Lead Leg Toe Off" : [elb['to_time']['rear_elbow_jc_3d_y'],elb['to_time']['lead_elbow_jc_3d_y']],
                    "at Knee High"        : [elb['kh_time']['rear_elbow_jc_3d_y'],elb['kh_time']['lead_elbow_jc_3d_y']],
                    "at Fooc Contact"     : [elb['fc_time']['rear_elbow_jc_3d_y'],elb['fc_time']['lead_elbow_jc_3d_y']],
                    "at Ball Contact"     : [elb['bc_time']['rear_elbow_jc_3d_y'],elb['bc_time']['lead_elbow_jc_3d_y']],
                    }

    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Elbow Height (Loss of Space)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    # data_as_dict = {
    #                 "Segment"                     : ["Rear Elbow"                    , "Lead Elbow"],
    #                 "Max"                         : [elb['max']['rear_elbow_jc_3d_y'],elb['max']['lead_elbow_jc_3d_y'] ],
    #                 "Max Timing [BC - FC (100%)]" : [f"{round(100 * (elb['max_time']['rear_elbow_jc_3d_y'] - fc_time) / total_time,2)} %",f"{round(100 * (elb['max_time']['lead_elbow_jc_3d_y'] - fc_time) / total_time,2)} %" ],
    #                 }
    # pdf.set_y(118)
    # title_size = 8
    # pdf.set_font(family='Arial', style='B', size=title_size)
    # pdf.create_table(table_data = data_as_dict,
    #                 title = '',
    #                 title_size=10,
    #                 align_header = 'C',
    #                 align_data = 'C',
    #                 cell_width = [64,64,64],
    #                 x_start='C')
    
    # ===================================================================================
    # LEAD SHOULDER - LEAD ELBOW DISTANCE
    pdf.set_link(ch10_link, page = 7)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD SHOULDER - LEAD ELBOW DISTANCE [Check Dead Hands]', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The distance between the lead shoulder and lead elbow')
    pdf.image(f'figure/SHOULDER-HAND DISTANCE (AP).png',x=75,y=160,w=133,h=50)
    pdf.image(f'{logo_path}/shoulder_hand_ap.png',x=17,y=160,w=50,h=50)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['hand_shoulder_distance']],
                    "at Knee High"        : [ang['kh_time']['hand_shoulder_distance']],
                    "at Fooc Contact"     : [ang['fc_time']['hand_shoulder_distance']],
                    "at Ball Contact"     : [ang['bc_time']['hand_shoulder_distance']]
                    }
    pdf.set_y(221)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Hip - Elbow Loss of Space',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                    "Min"        : [ang['min']['hand_shoulder_distance']],
                    "Min Timing [BC - FC (100%)]" : [f"{round(100 * (ang['min_time']['hand_shoulder_distance'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(244)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')   
    # =====================================================================================
    # comment
    pdf.add_page()
    comment_path, _ = os.path.split(file)
    comment_path = os.path.join(Path(comment_path).parent, 'comment')
    
    pdf.set_link(ch11_link, page = 8)
    with open(f"{comment_path}/{name.split('_')[0]}_comment.txt", "rb") as fh:
        comment_txt = fh.read().decode('utf-8')

    pdf.set_y(20)
    pdf.set_font('kor', 'B', 20)
    pdf.cell(0, 0, 'FEEDBACK', ln = True, align = 'C')
    pdf.set_font('kor', 'B', 10)
    pdf.set_y(30)
    pdf.multi_cell(0, 8, comment_txt, ln = True, border='True')


    # figure_file = glob('figure/*png')
    # for figure in figure_file:
    #     os.remove(figure)
    
    pdf.output(f'{store}/report/{name}_{day}_baseball_report(upflit).pdf')

2it [00:17,  8.55s/it]


In [19]:
comment_txt['- KINEMATIC SEQUENCE & STRIDE LENGTH' in comment_txt]

' '

In [26]:
position_kinematic = comment_txt.find('- KINEMATIC SEQUENCE & STRIDE LENGTH')
position_grf_y = comment_txt.find('- GROUND REACTION FORCE (AP axis)')
position_grf_z = comment_txt.find('- GROUND REACTION FORCE (Vertical)')
position_xfactor = comment_txt.find('- HIP/SHOULDER SEPARATION')
position_elb_flx = comment_txt.find('- ELBOW FLEXION')
position_ser = comment_txt.find('- SHOULDER EXTERNAL ROTATION')
position_sho_ha = comment_txt.find('- SHOULDER HORIZONTAL ABDUCTION')
position_sho_abd = comment_txt.find('- SHOULDER ABDUCTION')
position_lead_leg_knee_ext_vel = comment_txt.find('- LEAD LEG KNEE EXTENSION ANGULAR VELOCITY')
position_lead_leg_knee_flx = comment_txt.find('- LEAD LEG KNEE FLEXION')
position_trunk_fwd_tilt = comment_txt.find('- TRUNK FORWARD TILT')
position_trunk_lat_tilt = comment_txt.find('- TRUNK LATERAL TILT')

Found at position 55


In [27]:
comment_txt[position_kinematic:position_grf_y]

'- KINEMATIC SEQUENCE & STRIDE LENGTH\n\n몸통 속도 좋다\n나쁘지 않다\n\n'

In [28]:
comment_txt[position_grf_y:position_grf_z]

'- GROUND REACTION FORCE (AP axis)\n\n나쁘지 안혹 나쁘지 않다.\n\n'